## Calculate general statistics

Use this section to have an overview of the dataset in terms of fields published, stages covered and key summary statistics.  You can add any additional queries as needed.  

###Fields published

Calculate the fields published

In [ ]:
%%sql fields_table <<

SELECT
  path, distinct_releases
FROM
  field_counts
WHERE
  release_type='compiled_release'

In [ ]:
fields_table

Save fields table in spreadsheet:

In [ ]:
save_dataframe_to_sheet(fields_table, 'fields')

### Stages covered

In [ ]:
%%sql stages <<

SELECT
    CASE WHEN paths.path = 'contracts/implementation' THEN
        'implementation'
    ELSE
        paths.path
    END AS stage,
    CASE WHEN paths.path IN ('planning', 'tender', 'contracts/implementation') THEN
        GREATEST (object_property, 0)
    ELSE
        GREATEST (array_count, 0)
    END AS object_count
FROM (
    SELECT
        unnest(ARRAY['planning', 'tender', 'awards', 'contracts', 'contracts/implementation']) AS path) AS paths
    LEFT JOIN (
        SELECT
            *
        FROM
            field_counts
        WHERE
            release_type = 'compiled_release'
            AND path IN ('planning', 'tender', 'awards', 'contracts', 'contracts/implementation')) AS field_counts USING (path)



In [ ]:
stages

In [ ]:
import plotly.express as px

fig = px.bar(stages, x='stage', y='object_count', title="Stages covered",
             template='plotly_white',
             text='object_count',
             labels={'stage': 'Stage',
                     'object_count': 'Number of releases'})
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=400, height=350, bargap=0.5)
fig.show()

### Number of tenders and awards by date

In [ ]:
%%sql dates <<
WITH tenders AS(
  SELECT
	  EXTRACT(YEAR from tenderperiod_startdate) as year,
	  COUNT(*) AS tenders
  FROM
	  tender_summary
  WHERE
    release_type='compiled_release'
  GROUP BY
	  year
  ORDER BY
    year),

awards AS(
  SELECT
		EXTRACT(YEAR from date) as year,
		COUNT(*) AS awards
	FROM
		awards_summary
WHERE
    release_type='compiled_release'
	GROUP BY
		year
	ORDER BY
		year
)

SELECT
  t.year,
  tenders,
  awards
FROM
  tenders t FULL JOIN awards a ON t.year=a.year


In [ ]:
dates

In [ ]:
final=pd.melt(dates, id_vars=["year"], value_vars=["tenders", "awards"])
fig = px.bar(final, x='year', y='value', color='variable', barmode="group" ,
             title="Tenders and awards by year", template='plotly_white', text='value',

             color_discrete_sequence=['#D6E100', '#6c75e1'],
             labels ={
                 'value': "Tenders and awards"
             })
fig.update_layout(width=600, height=350, bargap=0.5)

fig.show()

### Procurement methods used

In [ ]:
%%sql

SELECT
  procurementmethod,
  COUNT(DISTINCT (ocid)),
  ROUND((count(*) * 100.0 / sum(count(*)) over()),1) AS proportion
FROM
  tender_summary
WHERE
  release_type='compiled_release'
GROUP BY
  procurementmethod
ORDER BY
  proportion DESC

In [ ]:
%%sql

SELECT
  tender ->> 'procurementMethodDetails' as method,
  COUNT(DISTINCT (ocid)),
  ROUND((count(*) * 100.0 / sum(count(*)) over()),1) AS proportion
FROM
  tender_summary
WHERE
  release_type='compiled_release'
GROUP BY
  method
ORDER BY
  proportion DESC

### Number of procedures by buyer

In [ ]:
%%sql buyers <<

SELECT
  party-> 'name' AS "name",
  identifier as party_id,
	COUNT(DISTINCT ocid) AS total_tenders
FROM
	parties_summary
WHERE
  roles::text ilike '%%buyer%%'
AND
  release_type= 'compiled_release'
GROUP BY
		identifier,  	party-> 'name'
ORDER BY
	total_tenders DESC

In [ ]:
buyers.head(10)

In [ ]:
fig = px.bar(buyers.head(10), y='name', x='total_tenders',
              title="Top 10 buyers", template='plotly_white',
             text='total_tenders',
             labels={'total_tenders': 'Number of procedures',
                     'name': 'buyer'},
             color_discrete_sequence=['#D6E100', '#6c75e1'], orientation='h')
#fig.update_traces()
fig.update_layout(width=850, height=450, bargap=0.4,
yaxis={'categoryorder':'total ascending'})
fig.show()